
***EmoDiary : 스마트 감정 일기 도우미 서비스***

https://huggingface.co/spaces/dazzleun-7/Bigdatacapstone_24-2

- [부제 1] 감정 실시간 감지를 통한 맞춤형 플레이리스트 추천
- [부제 2] 일기 회고 콘텐츠 추천
---
- 오늘 하루 일상을 표현하는 얼굴 표정 + 텍스트를 받음
- 1)감정과 유사한 or 반대되는 음악을 추천해주고
- 2)생성형 AI는
    - 일기 모멘텀의 역할 : 지속적으로 일기를 작성할 수 있도록 동기부여와 흐름을 제공
       - 사용자의 감정 벡터를 기반으로 감정 캐릭터 이미지 제공
       - 사용자가 남긴 하루 기록에 대해 답장 제공
       - 내면을 들여다볼 수 있는 구체적인 회고 콘텐츠 제공
---

*  일기 쓰기는 개인이 자신의 생각과 감정을 깊이 탐구하고 이를 글로 표현함으로써 자기 성찰을 촉구하는 강력한 도구로 널리 인정받고 있음. 특히, 정서적으로 복잡한 상황을 글로 정리하는 과정에서 개인은 자신을 객관적으로 이해하고, 정서적 안정을 도모할 수 있음

    - (기타) 하지만, 현재 일기를 쓸 때 느끼는 어려운 점
        - 일기를 통한 감정 파악의 어려움
        - 일기의 소재를 떠올리기 어려움
        - 글을 작성하는 것에 대한 부담감

* 음악 감상은 개인의 정서 조절에 효과적이며, 자기 성찰 과정에서 감정 인식을 돕는 역할을 한다는 연구 결과가 보고됨. (https://s-space.snu.ac.kr/handle/10371/120423?utm_source=chatgpt.com)

* 이러한 이론적 근거를 토대로 본 프로젝트는 사용자 감정 분석을 기반으로 한
음악과 일기 작성 간의 심리적 효과를 연계하는 기술적 접근을 제안하여 사용자의 정서적 건강 증진과 자기 성찰을 지원할 수 있도록 한다. 또한 생성형 AI를 통해 단순히 '오늘의 기록'을 넘어 감정적 연결을 끌어내고, 회고 가이드라인을 제공하여 글을 쓴다는 것에 대한 부담감을 줄이며 자신을 더 잘 이해할 수 있도록 도움
---

#1.Colab 환경설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#필요 패키지 설치
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install numpy==1.23.1

#KoBERT 깃허브에서 불러오기
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

!pip install langchain==0.0.125 chromadb==0.3.14 pypdf==3.7.0 tiktoken==0.3.3
!pip install openai==0.28
!pip install gradio transformers torch opencv-python-headless

#gradio
!pip install --upgrade gradio

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

#  Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

# 2. 데이터 전처리

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/kobert.csv')
df

In [ ]:
#슬픔&상처&불안 -> 우울로 변경
df.loc[df['감정_대분류'] == '슬픔', '감정_대분류'] = '우울'
df.loc[df['감정_대분류'] == '상처', '감정_대분류'] = '우울'
df.loc[df['감정_대분류'] == '불안', '감정_대분류'] = '우울'

In [ ]:
# 당황 데이터 제거
df = df[df['감정_대분류'] != '당황'].reset_index(drop=True)
df['감정_대분류'].value_counts()

In [ ]:
df = df.reset_index(drop=True)
df = df[['감정_대분류', '사람문장1']]
df

In [ ]:
df['감정_대분류'].unique()

In [ ]:
# 6개의 감정 class → 숫자
df.loc[(df['감정_대분류'] == "기쁨"), '감정_대분류'] = 0  # 기쁨 → 0
df.loc[(df['감정_대분류'] == "즐거움"), '감정_대분류'] = 1  # 즐거움 → 1
df.loc[(df['감정_대분류'] == "사랑"), '감정_대분류'] = 2  # 사랑 → 2
df.loc[(df['감정_대분류'] == "분노"), '감정_대분류'] = 3  # 분노 → 3
df.loc[(df['감정_대분류'] == "우울"), '감정_대분류'] = 4  # 우울 → 4
df.loc[(df['감정_대분류'] == "외로움"), '감정_대분류'] = 5  # 외로움 → 5

In [ ]:
df['감정_대분류'].unique()

In [ ]:
data_list = []
for q, label in zip(df['사람문장1'], df['감정_대분류'])  : ## BERTDataset에 input으로 주기 위해 문장과 감정라벨로 이루어진 list append
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
print(len(data_list))
print(data_list[0])
print(data_list[140])
print(data_list[1534])
print(data_list[3000])
print(data_list[-1])

In [ ]:
df[250:600]

#3.데이터 분리


In [ ]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

print(len(dataset_train))
print(len(dataset_test))

In [ ]:
from collections import Counter
from sklearn.utils.class_weight import compute_sample_weight
from torch.utils.data import DataLoader, WeightedRandomSampler
import numpy as np
import pandas as pd
import torch

# dataset_train에서 레이블(두 번째 항목)을 추출하여 카운트
labels_train = [item[1] for item in dataset_train]
label_counts_train = Counter(labels_train)

# 레이블 분포 출력 --> train 데이터 내부에서 데이터 편향 문제 발생
print("Train 데이터 레이블 분포:")
print(label_counts_train)

In [ ]:
import random

# 언더샘플링 대상 레이블 설정 및 목표 샘플 수 정의
target_counts = {'4': 6000, '3': 4000, '0': 4000}

# 언더샘플링 적용할 데이터셋
final_dataset_train = []

# 레이블별로 데이터를 그룹화
data_by_label = {label: [] for label in target_counts.keys()}

# 레이블별로 데이터를 분류
for sentence, label in dataset_train:
    if label in target_counts:
        data_by_label[label].append((sentence, label))

# 다수 클래스는 목표 샘플 수만큼 무작위 샘플링
for label, items in data_by_label.items():
    target_count = target_counts[label]
    sampled_items = random.sample(items, target_count) if len(items) > target_count else items
    final_dataset_train.extend(sampled_items)

# 소수 클래스는 그대로 추가
for sentence, label in dataset_train:
    if label not in target_counts:
        final_dataset_train.append((sentence, label))

print("Undersampled Train Dataset Size:", len(final_dataset_train))

In [ ]:
# dataset_train에서 레이블(두 번째 항목)을 추출하여 카운트
labels_train = [item[1] for item in final_dataset_train]
label_counts_train = Counter(labels_train)

# 레이블 분포 출력
print("Train 데이터 레이블 분포:")
print(label_counts_train)

In [ ]:
labels_train_series = pd.Series(labels_train)
class_counts = labels_train_series.value_counts().to_dict()  # 딕셔너리 형태로 변경
num_samples = sum(class_counts.values())  # 총 샘플 수

# 클래스별 가중치 부여 (딕셔너리 형태로)
class_weights = {label: num_samples / count for label, count in class_counts.items()}

# 해당 데이터의 label에 해당되는 가중치 부여
weights = [class_weights[label] for label in labels_train]

# WeightedRandomSampler 정의
sampler = WeightedRandomSampler(weights=torch.DoubleTensor(weights), num_samples=len(weights))

class_weights

# 4. 데이터 변환 (토큰화, 정수 인코딩, 패딩)


In [ ]:
## Setting parameters
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 1e-5

In [ ]:
## BERT 스타일의 데이터 변환을 수행하는 클래스
## BERT 모델에 입력되는 문장 또는 문장 쌍을 적절한 형식으로 변환해 모델이 이해할 수 있도록 함


class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    # 입력으로 받은 tokenizerm 최대 시퀀스 길이, vocab, pad 및 pair 설정
    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    # 입력된 문장 또는 문장 쌍을 BERT 모델이 사용할 수 있는 형식으로 변환
    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [ ]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

## 주어진 데이터셋을 KoBERT 모델에 입력으로 사용할 수 있는 형식으로 변환 후 이를 토대로 텍스트 분류 작업을 수행할 수 있도록 함
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')


data_train = BERTDataset(final_dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

# KoBERTTokenizer를 사용해 한국어 BERT용 토크나이저와 BertModle을 사용해 사전 학습된 KoBERT 모델 불러옴
# nlp.vocab 사용해 BERT 모델의 어휘 불러옴
# 학습 및 테스트 데이터셋을 BERTDataset 클래스를 사용해 준비 --> 각각의 문장은 토큰화되고 어휘에 따라 숫자로 인덱싱됨

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2,sampler=sampler, shuffle=False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

#5-1. Kobert_softmax

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size=768,
                 num_classes=6,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        self.softmax = nn.Softmax(dim=1)  # Softmax로 변경
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(in_features=hidden_size, out_features=512),
            nn.Linear(in_features=512, out_features=num_classes),
        )

        # 정규화 레이어 추가 (Layer Normalization)
        self.layer_norm = nn.LayerNorm(768)

        # 드롭아웃
        self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        _, pooler = self.bert(input_ids=token_ids, token_type_ids=segment_ids.long(), attention_mask=attention_mask.float().to(token_ids.device))

        pooled_output = self.dropout(pooler)
        normalized_output = self.layer_norm(pooled_output)
        out = self.classifier(normalized_output)

        # LayerNorm 적용
        pooler = self.layer_norm(pooler)

        if self.dr_rate:
            pooler = self.dropout(pooler)

        logits = self.classifier(pooler)  # 분류를 위한 로짓 값 계산
        probabilities = self.softmax(logits)  # Softmax로 각 클래스의 확률 계산
        return probabilities  # 각 클래스에 대한 확률 반환


In [ ]:
#정의한 모델 불러오기
model = BERTClassifier(bertmodel,dr_rate=0.4).to(device)
#model = BERTClassifier(bertmodel,  dr_rate=0.5).to('cpu')

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
train_dataloader

# 6. 모델 학습 및 평가

In [ ]:
from sklearn.metrics import f1_score

# 각 에포크의 F1 스코어를 저장할 리스트 초기화
train_f1_scores = []
test_f1_scores = []

# 모델 훈련 및 평가
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    total_loss = 0.0
    all_train_preds = []
    all_train_labels = []
    all_test_preds = []
    all_test_labels = []
    train_losses = []
    train_accuracies = []
    test_accuracies =[]
    avg_test_acc=0.0
    model.train()

    # 훈련 데이터 학습
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        total_loss += loss.item()
        train_acc += calc_accuracy(out, label)

        # 예측값과 실제 라벨 저장
        _, preds = torch.max(out, dim=1)
        all_train_preds.extend(preds.cpu().numpy())
        all_train_labels.extend(label.cpu().numpy())

    # 에포크별 평균 손실 및 정확도 계산
    avg_train_loss = total_loss / len(train_dataloader)
    avg_train_acc = train_acc / len(train_dataloader)
    train_losses.append(avg_train_loss)
    train_accuracies.append(avg_train_acc)

    # F1 스코어 계산
    train_f1 = f1_score(all_train_labels, all_train_preds, average='macro')
    train_f1_scores.append(train_f1)

    print(f"epoch {e+1} train loss {avg_train_loss} train acc {avg_train_acc} train f1 {train_f1}")

    # 검증 데이터 평가
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)

        # 예측값과 실제 라벨 저장
        _, preds = torch.max(out, dim=1)
        all_test_preds.extend(preds.cpu().numpy())
        all_test_labels.extend(label.cpu().numpy())

    # 에포크별 테스트 정확도 및 F1 스코어 계산
    avg_test_acc = test_acc / len(test_dataloader)
    test_accuracies.append(avg_test_acc)
    test_f1 = f1_score(all_test_labels, all_test_preds, average='macro')
    test_f1_scores.append(test_f1)

    print(f"epoch {e+1} test acc {avg_test_acc} test f1 {test_f1}")


In [ ]:
# 저장 경로 설정 (Google Drive 내 원하는 폴더로 변경 가능)
#save_path = '/content/drive/MyDrive/model_weights_softmax.pth'

# 모델 상태_dict 저장
#torch.save(model.state_dict(), save_path)
#print(f'Model saved to {save_path}')

In [ ]:
# 모델 저장 경로
model_save_path = '/content/drive/MyDrive/model_weights_softmax(model).pth'

# 모델 전체 저장
torch.save(model, model_save_path)

# 7. 사용자 입력 문장


In [ ]:
#테스트
#토큰화
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
#tok = tokenizer.tokenize

sentence_emotions = []

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model (token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            #emotions = [round(value.item() *100,2) for value in i]
            emotions = [value.item() for value in i]
            #print(emotions)
            sentence_emotions.append(emotions)
        print(sentence_emotions)

#질문 무한반복하기! 0 입력시 종료
sentence = input("하고싶은 말을 입력해주세요 : ")
predict(sentence)
#end = 1
#while end == 1 :
 #   sentence = input("하고싶은 말을 입력해주세요 : ")
 #   if sentence == '0' :
 #       break
 #   predict(sentence)
    #print("\n")

In [ ]:
sentence_emotions

# 8. 사용자 표정 인식

### OpenCV 이용해서 실시간 사용자 이미지 데이터 받아오기

In [ ]:
pip  install  git+https://github.com/openai/CLIP.git  scikit-image  matplotlib

In [ ]:
!pip install opencv-python
import cv2

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

### Clip 모델 사용하여 image classification

* 사전 훈련이나 클래스의 사전 레이블이 지정되지 않은 데이터여도 완벽한 이미지 분류 성능을 달성함 (Zero-shot)    



In [ ]:
from transformers.utils import logging

logging.set_verbosity_error()

In [ ]:
# Load model directly
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification

processor = AutoProcessor.from_pretrained("openai/clip-vit-large-patch14")
model = AutoModelForZeroShotImageClassification.from_pretrained("openai/clip-vit-large-patch14")

In [ ]:
from transformers import AutoProcessor

print(f"processor details for preprocessing: \n {processor}")

In [ ]:
from PIL import Image

In [ ]:
images = Image.open("/content/photo.png")

In [ ]:
labels = ['a photo of a happy face', 'a photo of a joyful face', 'a photo of a loving face', 'a photo of a angry face',  'a photo of a melancholic face',  'a photo of a lonely face']

In [ ]:
inputs = processor(text = labels,
                  images = images,
                  return_tensors = "pt",
                  padding = True)

In [ ]:
print(f"details of inputs: \n {inputs}")

In [ ]:
outputs = model(**inputs)
print(f"outputs: \n {outputs}")

In [ ]:
outputs.logits_per_image

In [ ]:
probs = outputs.logits_per_image.softmax(dim=1)[0]  # softmax로 변환

print(f"probs : {probs}")

In [ ]:
probs = list(probs)
for i in range(len(labels)):
  print(f"label: {labels[i]} - probability of {probs[i].item():.4f}")

# 9. 최종 사용자 감정 벡터 출력 (텍스트 + 표정)


*   텍스트 기반 감정 분석의 신뢰도가 안면 감정 분석보다 더 높다는 연구 결과를 반영하여 가중치를 각각 0.7과 0.3으로 설정함

*   v combined =0.7*vtext+0.3*vimage​






In [ ]:
# 얼굴 표정 기반 감정 벡터 추출
image_emotions = [float(prob.item()) for prob in probs]
image_emotions

In [ ]:
# 텍스트 기반 감정 벡터 추출
sentence_emotions = [item for sublist in sentence_emotions for item in sublist]
sentence_emotions

In [ ]:
image_emotions = np.array(image_emotions)
sentence_emotions = np.array(sentence_emotions)

In [ ]:
final_user_emotions = image_emotions * 0.3 + sentence_emotions * 0.7
final_user_emotions

#10. 모델 불러오기

In [ ]:
model = torch.load('/content/drive/MyDrive/model_weights_softmax(model).pth의 사본')
model.eval()

#11. 멜론데이터 감정 벡터 불러오기


In [ ]:
# 전처리 전 멜론 데이터
melon_data = pd.read_csv('/content/drive/MyDrive/melon_data.csv')
melon_data

In [ ]:
# 전처리 + 감정 벡터로 표현된 멜론 데이터
melon_emotions = pd.read_csv('/content/drive/MyDrive/melon_emotions_final.csv')
melon_emotions

In [ ]:
melon_emotions

In [ ]:
melon_emotions = pd.merge(melon_emotions, melon_data, left_on='Title', right_on='title', how='inner')
melon_emotions = melon_emotions[['singer', 'Title', 'genre','Emotions']]

In [ ]:
# [가수명, 곡명, 장르, 감정벡터]로 최종 정리된 멜론 데이터 프레임
melon_emotions

In [ ]:
# 멜론 데이터의 Title에서 중복되는 노래 제목 하나만 남기고 제거
melon_emotions = melon_emotions.drop_duplicates(subset='Title', keep='first')

In [ ]:
# 중복된 값이 포함된 행 전체 확인
duplicate_rows = melon_emotions[melon_emotions['Title'].duplicated(keep=False)]
print(duplicate_rows)

#12. 코사인 유사도

In [ ]:
import pandas as pd
import ast

melon_emotions['Emotions'] = melon_emotions['Emotions'].apply(lambda x: ast.literal_eval(x))

emotions = melon_emotions['Emotions'].to_list()

In [ ]:
print(emotions)

In [ ]:
print(final_user_emotions)

In [ ]:
print(type(final_user_emotions))
print(type(emotions))

In [ ]:
# 코사인 유사도 함수
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1 == 0 or norm_vec2 == 0:
        return np.nan  # 제로 벡터인 경우 NaN 반환
    return dot_product / (norm_vec1 * norm_vec2)

# 각 노래에 대한 코사인 유사도 계산
similarities = [cosine_similarity(final_user_emotions, song_vec) for song_vec in emotions]

# 유사도가 NaN이 아닌 인덱스만 필터링
valid_indices = [i for i, sim in enumerate(similarities) if not np.isnan(sim)]
filtered_similarities = [similarities[i] for i in valid_indices]

recommendations = np.argsort(filtered_similarities)[::-1]  # 높은 유사도 순으로 인덱스 정렬
print(recommendations)
# 결과를 데이터프레임으로 만들기
results_df = pd.DataFrame({
    'Singer' : melon_emotions['singer'].iloc[recommendations].values,
    'title' : melon_emotions['Title'].iloc[recommendations].values,
    'genre' : melon_emotions['genre'].iloc[recommendations].values,
    'Cosine Similarity': [similarities[idx] for idx in recommendations]
})

In [ ]:
results_df

In [ ]:
results_df.head(5)

In [ ]:
results_df.tail(5)

# 13. 플레이리스트 1차 필터링

In [ ]:
# 감정과 유사한 플레이리스트 (상위 5개)
similar_playlists = results_df.head(5)

# 감정과 반대되는 플레이리스트 (하위 5개)
dissimilar_playlists = results_df.tail(5)

# 사용자에게 선택지 보여주기
print("내 감정과 유사한 플레이리스트 (상위 5개):")
print(similar_playlists)

print("\n내 감정과 반대되는 플레이리스트 (하위 5개):")
print(dissimilar_playlists)

# 사용자 선호도 입력받기
print("\n어떤 플레이리스트를 듣고 싶으신가요?")
print("1. 내 감정과 유사한 플레이리스트")
print("2. 내 감정과 반대되는 플레이리스트")
choice = input("선택 (1 또는 2 입력): ")

# 14. 플레이리스트 2차 필터링


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import ast

# 가중치 값 설정
gamma = 0.3

# 초기 데이터 처리
if choice == '1':
    similar_playlists = pd.merge(similar_playlists, melon_emotions, left_on="title", right_on="Title", how="inner")
    similar_playlists = similar_playlists[["title", "Emotions", "singer"]]

    results = []
    seen_songs = set(similar_playlists["title"].values)  # 초기 seen_songs에 similar_playlists의 곡들을 추가

    # 사용자 감정 벡터
    user_emotion_vector = np.array(final_user_emotions).reshape(1, -1)

    for index, row in similar_playlists.iterrows():
        song_title = row["title"]
        song_singer = row["singer"]
        song_vector = np.array(row["Emotions"]).reshape(1, -1)

        song_results = []
        for i, emotion_vec in enumerate(emotions):
            emotion_title = melon_emotions.iloc[i]["Title"]
            emotion_singer = melon_emotions.iloc[i]["singer"]
            emotion_vec = np.array(emotion_vec).reshape(1, -1)

            # similar_playlists에 있는 곡과 seen_songs에 있는 곡은 제외
            if (
                emotion_title != song_title and
                emotion_title not in seen_songs
            ):
                try:
                    # 곡 간 유사도(Song-Song Similarity)
                    song_song_similarity = cosine_similarity(song_vector, emotion_vec)[0][0]

                    # 사용자 감정 벡터와의 유사도(User-Song Similarity)
                    user_song_similarity = cosine_similarity(user_emotion_vector, emotion_vec)[0][0]

                    # Final Score 계산
                    final_score = gamma * song_song_similarity + (1 - gamma) * user_song_similarity

                    song_results.append({
                        "Title": emotion_title,
                        "Singer": emotion_singer,
                        "Song-Song Similarity": song_song_similarity,
                        "User-Song Similarity": user_song_similarity,
                        "Final Score": final_score
                    })
                except ValueError as e:
                    print(f"Error with {song_title} vs {emotion_title}: {e}")
                    continue

        # Final Score를 기준으로 상위 3곡 선택
        song_results = sorted(song_results, key=lambda x: x["Final Score"], reverse=True)[:3]
        seen_songs.update([entry["Title"] for entry in song_results])

        results.append({"Song Title": song_title, "Singer": song_singer, "Top 3 Similarities": song_results})

    # 결과 출력
    for result in results:
        print(f"{result['Singer']} - {result['Song Title']}")
        for entry in result["Top 3 Similarities"]:
            print(f"{entry['Singer']} - {entry['Title']} : Final Score {entry['Final Score']:.4f}")
            print(f"  (Song-Song Similarity: {entry['Song-Song Similarity']:.4f}, User-Song Similarity: {entry['User-Song Similarity']:.4f})")
        print("-" * 30)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import ast

# 가중치 값 설정
gamma = 0.3

# 초기 데이터 처리
if choice == '2':
    dissimilar_playlists = pd.merge(dissimilar_playlists, melon_emotions, left_on="title", right_on="Title", how="inner")
    dissimilar_playlists = dissimilar_playlists[["title", "Emotions", "singer"]]

    results = []
    seen_songs = set()

    # 사용자 감정 벡터
    user_emotion_vector = np.array(final_user_emotions).reshape(1, -1)

    for index, row in dissimilar_playlists.iterrows():
        song_title = row["title"]
        song_singer = row["singer"]
        song_vector = np.array(row["Emotions"]).reshape(1, -1)

        song_results = []
        for i, emotion_vec in enumerate(emotions):
            emotion_title = melon_emotions.iloc[i]["Title"]
            emotion_singer = melon_emotions.iloc[i]["singer"]
            emotion_vec = np.array(emotion_vec).reshape(1, -1)

            if (
                emotion_title != song_title and
                emotion_title not in dissimilar_playlists["title"].values and
                emotion_title not in seen_songs
            ):
                try:
                    # 곡 간 유사도(Song-Song Similarity)
                    song_song_similarity = cosine_similarity(song_vector, emotion_vec)[0][0]

                    # 사용자 감정 벡터와의 반대 유사도(User-Song Dissimilarity)
                    opposite_user_song_similarity = 1 - cosine_similarity(user_emotion_vector, emotion_vec)[0][0]

                    # Final Score 계산
                    final_score = gamma * song_song_similarity + (1 - gamma) * opposite_user_song_similarity

                    song_results.append({
                        "Title": emotion_title,
                        "Singer": emotion_singer,
                        "Song-Song Similarity": song_song_similarity,
                        "User-Song Dissimilarity": opposite_user_song_similarity,
                        "Final Score": final_score
                    })
                except ValueError as e:
                    print(f"Error with {song_title} vs {emotion_title}: {e}")
                    continue

        # Final Score를 기준으로 상위 3곡 선택 (값이 큰 곡이 반대되는 곡)
        song_results = sorted(song_results, key=lambda x: x["Final Score"], reverse=True)[:3]
        seen_songs.update(entry["Title"] for entry in song_results)

        results.append({"Song Title": song_title, "Singer": song_singer, "Top 3 Similarities": song_results})

    # 결과 출력
    for result in results:
        print(f"{result['Singer']} - {result['Song Title']}")
        for entry in result["Top 3 Similarities"]:
            print(f"{entry['Singer']} - {entry['Title']} : Final Score {entry['Final Score']:.4f}")
            print(f'  (Song-Song Similarity: {entry["Song-Song Similarity"]:.4f}, User-Song Dissimilarity: {entry["User-Song Dissimilarity"]:.4f})')
        print("-" * 30)

# 15. 최종 플레이리스트 산출


In [ ]:
import pandas as pd

# 데이터프레임 변환을 위한 리스트 생성
df_rows = []

for result in results:
    song_title = result['Song Title']
    song_singer = result['Singer']
    main_song_info = f"{song_singer} - {song_title}"

    for entry in result["Top 3 Similarities"]:
        combined_info = f"{entry['Singer']} - {entry['Title']}"
        df_rows.append({"1st 추천 플레이리스트": main_song_info, "2nd 추천 플레이리스트": combined_info})

# 데이터프레임 생성
final_music_playlist_recommendation = pd.DataFrame(df_rows)

# 곡 제목 그룹화하여 첫 번째 행에만 곡 제목 표시
final_music_playlist_recommendation["1st 추천 플레이리스트"] = final_music_playlist_recommendation.groupby("1st 추천 플레이리스트")["1st 추천 플레이리스트"].transform(
    lambda x: [x.iloc[0]] + [""] * (len(x) - 1)
)

final_music_playlist_recommendation

# 16. 생성형 AI

*api 비공개

In [ ]:
pip install openai==0.28

In [ ]:
import openai
print(openai.__version__)

### 1. 감정 표현 일러스트 캐릭터 이미지 생성



In [ ]:
sentence = '요즘 마음이 너무 무겁고 답답해. 가끔은 아무것도 하고 싶지 않고 그냥 누워만 있고 싶어. 특히 어제 친구와의 작은 말다툼이 계속 마음에 남아서 그런지, 내가 뭔가 잘못했나 싶은 생각이 들더라. 그 친구가 얼마나 소중한지 알기에 더 마음이 아프고 미안한데, 사과를 하려고 하면 무슨 말을 해야 할지 몰라서 주저하게 돼. 나도 내 마음이 왜 이러는지 모르겠어.'

In [ ]:
emotion_labels = ['기쁨', '즐거움', '사랑', '분노', '우울', '외로움']

In [ ]:
sentence_emotions = [0.00859011, 0.0011389 , 0.00697977, 0.06076408, 0.76232639, 0.16020081]

In [ ]:
def get_dominant_emotion(sentence_emotions, emotion_labels):
    """
    감정 벡터값에서 가장 높은 값을 가진 감정의 라벨을 반환하는 함수.

    Args:
        sentence_emotions (list of float): 감정 벡터값 리스트.
        emotion_labels (list of str): 각 감정 벡터에 대응되는 감정 라벨 리스트.

    Returns:
        str: 가장 높은 감정 벡터값에 대응하는 감정 라벨.
    """
    # 가장 높은 감정 벡터값의 인덱스 찾기
    max_index = np.argmax(sentence_emotions)

    # 해당 인덱스의 감정 라벨 반환
    return emotion_labels[max_index]

In [ ]:
import numpy as np

dominant_emotion = get_dominant_emotion(sentence_emotions, emotion_labels)
print(dominant_emotion)

In [ ]:
sentence

In [ ]:
dominant_emotion

In [ ]:
# 이미지 생성 요청
response = openai.Image.create(
    model="dall-e-3",  # 최신 DALL·E 모델 사용
    prompt=(
        f"{sentence}을 반영해서 {dominant_emotion} 감정을 표현하는 3D 스타일의 일러스트 캐릭터를 그려줘. "
        "캐릭터는 부드럽고 둥근 디자인에 표정이 감정을 잘 드러내야 해. "
        "감정을 시각적으로 표현할 수 있는 소품이나 작은 상징 (예: 분노는 불꽃)을 포함해줘. "
        "감정의 분위기를 반영하는 선명하고 깨끗한 색상을 사용하고, 캐릭터가 역동적이고 재미있는 자세를 취할 수 있도록 해줘. "
        "이미지에는 하나의 캐릭터만 나오게 해줘."
        "배경은 단순하고 밝은 색상으로 설정해서 캐릭터가 강조될 수 있도록 해줘."
    ),
    size="1024x1024",
    n=1
)

# 생성된 이미지 URL 출력
image_url = response.data[0].url
print("Generated Image URL:", image_url)


In [ ]:
response = openai.Image.create(
    model="dall-e-3",
    prompt=(
        f"{sentence} 내용을 바탕으로, 어린아이가 그린 듯한 파스텔 톤의 그림 일기를 만들어줘. "
        "그림은 색연필이나 크레용을 사용한 것처럼 부드럽고 질감이 느껴지도록 표현해줘. "
        "그림은 단순하면서도 감정을 잘 표현해야 하고, 글자나 텍스트는 일절 포함하지 말아줘. "
        "배경은 심플하고 깔끔하게 처리해서 주요 내용이 강조될 수 있도록 해줘."
    ),
    size="1024x1024",
    n=1
)

image_url = response.data[0].url
print(image_url)


### 2. 일기 모멘텀 : 추천된 일기 콘텐츠와 함께하는 오늘의 회고

In [ ]:
# 챗봇 스타일 옵션 제공
options = {
    1: "친근한",  # 부담 없이 편안한 친구 같은 대화
    2: "MZ세대",  # 트렌디한 유행어와 짧고 재미있는 대화
    3: "유머러스한",  # 웃음과 위트를 담은 대화
    4: "현명한 조언자", # 따뜻한 격려 및 조언을 담은 대화
    5: "문학적 감성",  # 시적이고 은유적인 표현을 담은 대화
}

# 옵션 출력
print("당신의 일기를 함께 작성하고 싶은 챗봇 스타일을 선택하세요:")
for key, value in options.items():
    print(f"{key}. {value}")

# 사용자 입력 처리
while True:
    try:
        # 사용자 번호 입력
        selection = int(input("번호를 입력하세요: "))

        # 유효한 번호인지 확인
        if selection in options:
            style = options[selection]
            print(f"선택한 스타일: {style}")
            break
        else:
            print("유효한 번호를 입력해주세요.")
    except ValueError:
        print("숫자를 입력해주세요.")

In [ ]:
# ChatGPT API를 활용한 일기 코멘트와 모멘텀 추천 로직

# 1. 일기에 대한 짧은 코멘트 생성
system_prompt_comment = f"너는 사용자가 작성한 일기에 대해 짧은 코멘트를 제공하는 {style} 챗봇이야."
response_comment = openai.ChatCompletion.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": system_prompt_comment},
            {"role": "user", "content": sentence}],
        temperature = 1,
        presence_penalty = 1,
        frequency_penalty = 1,
        n = 1
    )

# 코멘트 출력
comment = response_comment.choices[0].message.content
print(comment)

In [ ]:
import openai

system_prompt_momentum = f"너는 일기 제목과 오늘 하루를 돌아볼 수 있도록 실질적인 일기 주제를 4-5개 정도 추천해주는 {style}의 일기 컨텐츠를 제공하는 챗봇이야."

## 초기 사용자 일기

# 첫 번째 대화
def get_initial_response(sentence):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": system_prompt_momentum},
                {"role": "user", "content": sentence}
            ],
            temperature = 1,
            presence_penalty = 1,
            frequency_penalty = 1,
            n = 1
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"오류가 발생했습니다: {e}"

from datetime import datetime

def get_current_date():
    today = datetime.now()
    return today.strftime("%Y-%m-%d")


get_initial_response(sentence)

#17. Gradio (hugging face 배포 ver.)

In [ ]:
import numpy as np
import pandas as pd
import requests
from PIL import Image
import torch
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification
import gradio as gr
import openai
from sklearn.metrics.pairwise import cosine_similarity
import ast

In [ ]:
# gradio final ver ----------------------------

###### 기본 설정 ######
# OpenAI API 키 설정


# 모델 및 프로세서 로드
processor = AutoProcessor.from_pretrained("openai/clip-vit-large-patch14")
model_clip = AutoModelForZeroShotImageClassification.from_pretrained("openai/clip-vit-large-patch14")
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

# 예측 레이블
labels = ['a photo of a happy face', 'a photo of a joyful face', 'a photo of a loving face',
          'a photo of an angry face', 'a photo of a melancholic face', 'a photo of a lonely face']

###### 얼굴 감정 벡터 예측 함수 ######
def predict_face_emotion(image):
    # 이미지가 None이거나 잘못된 경우
    if image is None:
        return np.zeros(len(labels))  # 빈 벡터 반환

    # PIL 이미지를 RGB로 변환
    image = image.convert("RGB")

    # CLIP 모델의 processor를 이용한 전처리
    inputs = processor(text=labels, images=image, return_tensors="pt", padding=True)

    # pixel_values가 4차원인지 확인 후 강제 변환
    pixel_values = inputs["pixel_values"]  # (batch_size, channels, height, width)

    # CLIP 모델 예측: forward에 올바른 입력 전달
    with torch.no_grad():
        outputs = model_clip(pixel_values=pixel_values, input_ids=inputs["input_ids"])

    # 확률값 계산
    probs = outputs.logits_per_image.softmax(dim=1)[0]
    return probs.numpy()

###### 텍스트 감정 벡터 예측 함수 ######
sentence_emotions = []

def predict_text_emotion(predict_sentence):

    if not isinstance(predict_sentence, str):
        predict_sentence = str(predict_sentence)

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=1, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        for i in out:
            logits = i.detach().cpu().numpy()
            emotions = [value.item() for value in i]
            sentence_emotions.append(emotions)
    return sentence_emotions[0]  # 최종 리스트 반환

###### 최종 감정 벡터 계산 ######
def generate_final_emotion_vector(diary_input, image_input):
    # 텍스트 감정 벡터 예측
    text_vector = predict_text_emotion(diary_input)
    # 얼굴 감정 벡터 예측
    image_vector = predict_face_emotion(image_input)
    text_vector = np.array(text_vector, dtype=float)
    image_vector = np.array(image_vector, dtype=float)

    print(text_vector)
    print(image_vector)

    # 최종 감정 벡터 가중치 적용
    return (text_vector * 0.7) + (image_vector * 0.3)

####### 코사인 유사도 함수 ######
def cosine_similarity_fn(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1 == 0 or norm_vec2 == 0:
        return np.nan  # 제로 벡터인 경우 NaN 반환
    return dot_product / (norm_vec1 * norm_vec2)


####### 이미지 다운로드 함수 (PIL 객체 반환) ######
def download_image(image_url):
    try:
        response = requests.get(image_url)
        response.raise_for_status()
        return Image.open(requests.get(image_url, stream=True).raw)
    except Exception as e:
        print(f"이미지 다운로드 오류: {e}")
        return None

# 스타일 옵션
options = {
    1: "🌼 친근한",
    2: "🔥 트렌디한 MZ세대",
    3: "😄 유머러스한 장난꾸러기",
    4: "🧘 차분한 명상가",
    5: "🎨 창의적인 예술가",
}

# 일기 분석 함수
def chatbot_diary_with_image(style_option, diary_input, image_input, playlist_input):

    style = options.get(int(style_option.split('.')[0]), "🌼 친근한")

    # GPT 응답 (일기 코멘트)
    try:
        response_comment = openai.ChatCompletion.create(
            model="gpt-4-turbo",
            messages=[{"role": "system", "content": f"너는 {style} 챗봇이야."}, {"role": "user", "content": diary_input}],
        )
        comment = response_comment.choices[0].message.content
    except Exception as e:
        comment = f"💬 오류: {e}"

    # GPT 기반 일기 주제 추천
    try:
        topics = get_initial_response(style_option, diary_input)
    except Exception as e:
        topics = f"📝 주제 추천 오류: {e}"

    # DALL·E 3 이미지 생성 요청 (3D 스타일 캐릭터)
    try:
        response = openai.Image.create(
            model="dall-e-3",
            prompt=(
                  f"{diary_input}를 반영해서 감정을 표현하는 3D 스타일의 일러스트 캐릭터를 그려줘. "
                  "캐릭터는 부드럽고 둥근 디자인에 표정이 감정을 잘 드러내야 해. "
                  "감정을 시각적으로 표현할 수 있는 소품이나 작은 상징을 포함해줘. "
                  "감정의 분위기를 반영하는 선명하고 깨끗한 색상을 사용하고, 캐릭터가 역동적이고 재미있는 자세를 취할 수 있도록 해줘. "
                  "이미지에는 하나의 캐릭터만 나오게 해줘."
                  "배경은 단순하고 밝은 색상으로 설정해서 캐릭터가 강조될 수 있도록 해줘."
            ),
            size="1024x1024",
            n=1
        )
        # URL 가져오기 및 다운로드
        image_url = response['data'][0]['url']
        print(f"Generated Image URL: {image_url}")  # URL 확인
        image = download_image(image_url)
    except Exception as e:
        print(f"이미지 생성 오류: {e}")  # 오류 상세 출력
        image = None

    # 사용자 최종 감정 벡터
    final_user_emotions = generate_final_emotion_vector(diary_input,image_input)

    # 각 노래에 대한 코사인 유사도 계산
    similarities = [cosine_similarity_fn(final_user_emotions, song_vec) for song_vec in emotions]

    #유효한 유사도 필터링
    valid_indices = [i for i, sim in enumerate(similarities) if not np.isnan(sim)]
    filtered_similarities = [similarities[i] for i in valid_indices]

    recommendations = np.argsort(filtered_similarities)[::-1]  # 높은 유사도 순으로 정렬
    results_df = pd.DataFrame({
    'Singer' : melon_emotions['singer'].iloc[recommendations].values,
    'title' : melon_emotions['Title'].iloc[recommendations].values,
    'genre' : melon_emotions['genre'].iloc[recommendations].values,
    'Cosine Similarity': [similarities[idx] for idx in recommendations]
    })

    # 가중치 값 설정
    gamma = 0.3

    similar_playlists = results_df.head(5)
    similar_playlists = pd.merge(similar_playlists, melon_emotions, left_on="title", right_on="Title", how="inner")
    similar_playlists = similar_playlists[["title", "Emotions", "singer"]]

    dissimilar_playlists = results_df.tail(5)
    dissimilar_playlists = pd.merge(dissimilar_playlists, melon_emotions, left_on="title", right_on="Title", how="inner")
    dissimilar_playlists = dissimilar_playlists[["title", "Emotions", "singer"]]

    #감정과 유사한 플레이리스트
    if playlist_input == '비슷한':
      results = []
      seen_songs = set(similar_playlists["title"].values)  # 초기 seen_songs에 similar_playlists의 곡들을 추가

      # 사용자 감정 벡터
      user_emotion_vector = generate_final_emotion_vector(diary_input, image_input).reshape(1, -1)

      for index, row in similar_playlists.iterrows():
          song_title = row["title"]
          song_singer = row["singer"]
          song_vector = np.array(row["Emotions"]).reshape(1, -1)

          song_results = []
          for i, emotion_vec in enumerate(emotions):
              emotion_title = melon_emotions.iloc[i]["Title"]
              emotion_singer = melon_emotions.iloc[i]["singer"]
              emotion_vec = np.array(emotion_vec).reshape(1, -1)

              # similar_playlists에 있는 곡과 seen_songs에 있는 곡은 제외
              if (
                  emotion_title != song_title and
                  emotion_title not in seen_songs
              ):
                  try:
                      # 곡 간 유사도(Song-Song Similarity)
                      song_song_similarity = cosine_similarity(song_vector, emotion_vec)[0][0]

                      # 사용자 감정 벡터와의 유사도(User-Song Similarity)
                      user_song_similarity = cosine_similarity(user_emotion_vector, emotion_vec)[0][0]

                      # Final Score 계산
                      final_score = gamma * song_song_similarity + (1 - gamma) * user_song_similarity

                      song_results.append({
                          "Title": emotion_title,
                          "Singer": emotion_singer,
                          "Song-Song Similarity": song_song_similarity,
                          "User-Song Similarity": user_song_similarity,
                          "Final Score": final_score
                      })
                  except ValueError as e:
                      print(f"Error with {song_title} vs {emotion_title}: {e}")
                      continue

          # Final Score를 기준으로 상위 3곡 선택
          song_results = sorted(song_results, key=lambda x: x["Final Score"], reverse=True)[:3]
          seen_songs.update([entry["Title"] for entry in song_results])

          results.append({"Song Title": song_title, "Singer": song_singer, "Top 3 Similarities": song_results})

      # 결과 출력
      for result in results:
          print(f"{result['Singer']} - {result['Song Title']}")
          for entry in result["Top 3 Similarities"]:
              print(f"{entry['Singer']} - {entry['Title']} : Final Score {entry['Final Score']:.4f}")
              print(f"  (Song-Song Similarity: {entry['Song-Song Similarity']:.4f}, User-Song Similarity: {entry['User-Song Similarity']:.4f})")
          print("-" * 30)

    #반대 플레이리스트
    if playlist_input == '상반된':
      results = []
      seen_songs = set()

      # 사용자 감정 벡터
      user_emotion_vector = generate_final_emotion_vector(diary_input, image_input).reshape(1, -1)

      for index, row in dissimilar_playlists.iterrows():
          song_title = row["title"]
          song_singer = row["singer"]
          song_vector = np.array(row["Emotions"]).reshape(1, -1)

          song_results = []
          for i, emotion_vec in enumerate(emotions):
              emotion_title = melon_emotions.iloc[i]["Title"]
              emotion_singer = melon_emotions.iloc[i]["singer"]
              emotion_vec = np.array(emotion_vec).reshape(1, -1)

              if (
                  emotion_title != song_title and
                  emotion_title not in dissimilar_playlists["title"].values and
                  emotion_title not in seen_songs
              ):
                  try:
                      # 곡 간 유사도(Song-Song Similarity)
                      song_song_similarity = cosine_similarity(song_vector, emotion_vec)[0][0]

                      # 사용자 감정 벡터와의 반대 유사도(User-Song Dissimilarity)
                      opposite_user_song_similarity = 1 - cosine_similarity(user_emotion_vector, emotion_vec)[0][0]

                      # Final Score 계산
                      final_score = gamma * song_song_similarity + (1 - gamma) * opposite_user_song_similarity

                      song_results.append({
                          "Title": emotion_title,
                          "Singer": emotion_singer,
                          "Song-Song Similarity": song_song_similarity,
                          "User-Song Dissimilarity": opposite_user_song_similarity,
                          "Final Score": final_score
                      })
                  except ValueError as e:
                      print(f"Error with {song_title} vs {emotion_title}: {e}")
                      continue

          # Final Score를 기준으로 상위 3곡 선택 (값이 큰 곡이 반대되는 곡)
          song_results = sorted(song_results, key=lambda x: x["Final Score"], reverse=True)[:3]
          seen_songs.update(entry["Title"] for entry in song_results)

          results.append({"Song Title": song_title, "Singer": song_singer, "Top 3 Similarities": song_results})

      # 결과 출력
      for result in results:
          print(f"{result['Singer']} - {result['Song Title']}")
          for entry in result["Top 3 Similarities"]:
              print(f"{entry['Singer']} - {entry['Title']} : Final Score {entry['Final Score']:.4f}")
              print(f'  (Song-Song Similarity: {entry["Song-Song Similarity"]:.4f}, User-Song Dissimilarity: {entry["User-Song Dissimilarity"]:.4f})')
          print("-" * 30)
    # 데이터프레임 변환을 위한 리스트 생성
    df_rows = []

    for result in results:
        song_title = result['Song Title']
        song_singer = result['Singer']
        main_song_info = f"{song_singer} - {song_title}"

        for entry in result["Top 3 Similarities"]:
            combined_info = f"{entry['Singer']} - {entry['Title']}"
            df_rows.append({"1st 추천 플레이리스트": main_song_info, "2nd 추천 플레이리스트": combined_info})

    # 데이터프레임 생성
    final_music_playlist_recommendation = pd.DataFrame(df_rows)

    # 곡 제목 그룹화하여 첫 번째 행에만 곡 제목 표시
    final_music_playlist_recommendation["1st 추천 플레이리스트"] = final_music_playlist_recommendation.groupby("1st 추천 플레이리스트")["1st 추천 플레이리스트"].transform(
        lambda x: [x.iloc[0]] + [""] * (len(x) - 1)
    )

    return final_music_playlist_recommendation, comment, topics, image

# 일기 주제 추천 함수
def get_initial_response(style, sentence):
    style = options.get(int(style.split('.')[0]), "🌼 친근한")
    system_prompt_momentum = (
        f"너는 {style}의 챗봇이야. 사용자가 작성한 일기를 바탕으로 생각을 정리하고 내면을 돌아볼 수 있도록 "
        "도와주는 구체적인 일기 콘텐츠나 질문 4-5개를 추천해줘."
    )
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": system_prompt_momentum},
                {"role": "user", "content": sentence}
            ],
            temperature=1
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"📝 주제 추천 오류: {e}"

# Gradio 인터페이스
with gr.Blocks() as app:
    gr.Markdown("# ✨ 스마트 감정 일기 서비스 ✨\n\n 오늘의 하루를 기록하면, 그에 맞는 플레이리스트와 일기 회고 콘텐츠를 자동으로 생성해드립니다!")
    with gr.Row():
        with gr.Column():
            chatbot_style = gr.Radio(
                choices=[f"{k}. {v}" for k, v in options.items()],
                label="🤖 원하는 챗봇 스타일 선택"
            )
            diary_input = gr.Textbox(label="📜 오늘의 하루 기록하기", placeholder="ex)오늘 소풍가서 맛있는 걸 많이 먹어서 엄청 신났어")
            image_input = gr.Image(type="pil", label="📷 얼굴 표정 사진 업로드")
            playlist_input = gr.Radio(["비슷한", "상반된"], label="🎧 오늘의 감정과 ㅇㅇ되는 플레이리스트 추천 받기")
            submit_btn = gr.Button("🚀 분석 시작")

        with gr.Column():
            output_playlist = gr.Dataframe(label="🎧 추천 플레이리스트 ")
            output_comment = gr.Textbox(label="💬 AI 코멘트")
            output_topics = gr.Textbox(label="📝 추천 일기 콘텐츠")
            output_image = gr.Image(label="🖼️ 생성된 오늘의 감정 캐릭터", type="pil", width=512, height=512)

    # 버튼 클릭 이벤트 연결
    submit_btn.click(
        fn=chatbot_diary_with_image,
        inputs=[chatbot_style, diary_input, image_input, playlist_input],
        outputs=[output_playlist, output_comment, output_topics, output_image]
    )

# 앱 실행
app.launch(debug=True)